# Modelando os tópicos da RBEF ao longo do tempo usando Latent Dirichlet Allocation 

## Imports


In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import os
from multiprocessing import Pool

import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

from utils.pre_processing import tokenizer, html_to_text
from utils.model import add_bigram, create_dictionary, create_corpus
from utils.model import ModelLDA, calc_coherence
from utils.plots import plot_frequent_words

## Leitura do banco de dados

In [2]:
df = pd.read_feather('scielo_dataset.feather')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1664 entries, 0 to 1663
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   titulo      1664 non-null   object
 1   seção       1664 non-null   object
 2   nova_seção  1664 non-null   object
 3   doi_id      1644 non-null   object
 4   texto_html  1664 non-null   object
dtypes: object(5)
memory usage: 65.1+ KB


In [4]:
df['nova_seção'].value_counts(normalize=True) * 100

nova_seção
artigos gerais                         48.497596
pesquisa em ensino de física           14.182692
produtos e materiais didáticos         12.139423
história da física e ciências afins     9.314904
outros                                  8.954327
seções especiais                        5.949519
SEM CATEGORIA                           0.961538
Name: proportion, dtype: float64

## Tokenizaçao e Lemmetização

In [ ]:
with Pool() as pool:
    # limpa o html dos textos.
    textos = list(tqdm(pool.imap(html_to_text, df['texto_html']), total=len(df)))
    
with Pool() as pool:
    # tokeniza e lemmetiza os textos.
    documentos = list(tqdm(pool.imap(tokenizer, textos), total=len(textos)))

  0%|          | 0/1664 [00:00<?, ?it/s]

  0%|          | 0/1664 [00:00<?, ?it/s]

## Bigram e criação do dicionário

In [ ]:
add_bigram(documentos, min_count=10)

In [ ]:
dicionario_sem_filtro = create_dictionary(documentos, filtrar=False)
dicionario = create_dictionary(documentos, n_abaixo=30, n_acima=0.5)

In [ ]:
plot_frequent_words(dicionario_sem_filtro, filename='dicionario_completo.png')

In [ ]:
plot_frequent_words(dicionario, filename='dicionario_filtro.png')

In [ ]:
corpus = create_corpus(dicionario, documentos)

In [ ]:
lda = ModelLDA(corpus, dicionario, chunksize=2000, iterations=100, passes=25)

In [ ]:
model = lda.run(8)

In [ ]:
model.print_topics()

In [ ]:
calc_coherence(model, documentos, dicionario, corpus, method='c_v')

In [ ]:
topicos = model.get_document_topics(corpus)
df['topicos'] = [max(topic, key=lambda x: x[1])[0] for topic in topicos]

In [ ]:
pd.crosstab(df['nova_seção'], df['topicos'])

In [ ]:
len(df.query('topicos.isin([5,3])'))